Nov 19
下面是找出来的几个候选  
pdfminer3k  
pdfminer.six  
pypdf  
pypdfocr 


In [1]:
# file paths
phone_numbers = ["./data/phone_numbers/070219_KV_BR_Telefonverzei.pdf",
                "./data/phone_numbers/52_pdf_wegweiser_schlaganfall.pdf",
                 "./data/phone_numbers/FL_SYB_BetriebsaerztlicherDienst_ID8414.pdf",
                 "./data/phone_numbers/Infoblatt_154.pdf"
                ]
scans = ["./data/scans/double_ocr.pdf",
         "./data/scans/single_ocr.pdf"]

isbn = ["./data/isbn/gelbe_seiten_2019-43.pdf"]

unit = ["./data/unit_conversion/si.txt"]


### 1-2 2 Methods of Converting PDF to Plain Text

### pdfminer.six
```
pip install pdfminer.six
```

In [82]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

def convert(case, fname, pages=None):
    if not pages: pagenums = set();
    else:         pagenums = set(pages);      
    manager = PDFResourceManager() 
    codec = 'utf-8'
    caching = True
    if case == 'text' :
        output = io.StringIO()
        converter = TextConverter(manager, output, codec=codec, laparams=LAParams())     
    if case == 'HTML' :
        output = io.BytesIO()
        converter = HTMLConverter(manager, output, codec=codec, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)   
    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    convertedPDF = output.getvalue()  
    infile.close(); converter.close(); output.close()
    return convertedPDF

#convertedPDF = convert('HTML', phone_numbers[0])
#print(convertedPDF)

### PDF TO Text
For Debian, Ubuntu, and friends
```
sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
```
For Fedora, Red Hat, and friends
```
sudo yum install gcc-c++ pkgconfig poppler-cpp-devel python3-devel
```
For Mac OS
```
brew install pkg-config poppler
```
then run
```
pip install pdftotext
```

In [83]:
import pdftotext

def get_text_pdftotext(pdf):
    with open(pdf, "rb") as f:
        pdf = pdftotext.PDF(f)
        return ("\n\n".join(pdf))
        #for page in pdf:
            #print(page)
#print(get_text_pdftotext(phone_numbers[1]))


## phone number format
### phone_numbers[0]
06221 / 56 8781  
06221 / 56 38099  
### phone_numbers[1]
0 62 21 – 56 78 42  en dash here  
0 72 61 – 66-0  
0 62 21 - 319 – 0  
0 72 61 – 19 29-2  
### phone_numbers[2]
06221 – 56 8966
### phone_numbers[3]
19292  
110  
56-7272  

前半段地区号： \+?\d+[[ ]?\d+]*[/|–|-]?[\s?\d+]*

### 1-2  4. Extract infomation from PDF

In [5]:
import re
re_email = re.compile("\w+@[\w-]+[\.\w+]+")
re_phone = re.compile("\+?\d+[\s?\d+]*[/|–|-]?[\s?\d+]*")
re_isbn = re.compile("\d*-\d*-\d*-\d*-\d*")
re_url = re.compile("https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}")
def extract_info(regex, files):
    for file in files:
        text = get_text_pdftotext(file)
        infos = regex.findall(text)
        #print(infos)
        for info in infos:
            print(info)
#extract_info(re_url, phone_numbers)

#### 4.(i) From the PDF files in the folder phone_numbers/ extract as many valid phone numbers as possible

In [ ]:
re_phone = re.compile("\+?\d+[\s?\d+]*[/|–|-]?[\s?\d+]*")
extract_info(re_phone, phone_numbers)

#### 4.(ii) Extract valid URLs and Email addresses from the files in the folder phone_numbers/.

In [ ]:
# extract emails
re_email = re.compile("\w+@[\w-]+[\.\w+]+")
extract_info(re_email, phone_numbers)
# extract urls
re_url = re.compile("https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}")
extract_info(re_url, phone_numbers)

#### 4.(iii) From the PDF file in the folder isbn/ extract all valid ISBN numbers.

In [ ]:
re_isbn = re.compile("\d*-\d*-\d*-\d*-\d*")
extract_info(re_isbn, phone_numbers)

#### 4.(iv) Convert values

In [29]:
def ml2l(ml_str):
    ml_str = ml_str.group()
    # extract the number part
    ml_str = re.search("([1-9][0-9]*(,[0-9]{3})*(\.[0-9]+)*)", ml_str).group()
    # get rid of , in large numbers  "2,500" => "2500"
    ml_str = ml_str.replace(',', '')
    ml_num = float(ml_str)/1000
    return format(ml_num, '.3f')

re_ml = re.compile("(([1-9][0-9]*(,[0-9]{3})*(\.[0-9]+)*)[ ]?(ml|milliliters))")
re_num = re.compile("([1-9][0-9]*(,[0-9]{3})*(\.[0-9]+)*)")
f = open(unit[0],'r')
content = f.read()
print(content)
ret = re.sub(r"(([1-9][0-9]*(,[0-9]{3})*(\.[0-9]+)*)[ ]?(ml|milliliters))", ml2l, content)
print(ret)

1337 ml
2,500 ml
12 milliliters
26 l
18,421.9 ml
8321ml
32 m

1.337
2.500
0.012
26 l
18.422
8.321
32 m



In [14]:
str_flt = "1223.123 "
print(float(str_flt))

1223.123
